In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [2]:
local_path = r"C:/Users/vishn/Desktop/College/2_AC2022_F_089.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [3]:
# Preview first page
data[0].page_content

'SUNSPOT DETECTION USING DEEP LEARNING TECHNIQUES\n\nPieter Swanepoel, Günther Richard Drevin, Roelf Du Toit Strauss and Petrus Johannes Steyn North-West University Potchefstroom, South Africa\n\nABSTRACT\n\nAccurate and automated monitoring of space weather has become increasingly important in our modern-day world, due to the reliance on critical infrastructure such as wireless communication, power grids, satellite communication, and aviation systems. Solar events such as solar flares and coronal mass ejections can disrupt or damage these system and sunspots are used as markers to determine solar activity through a manual process, thus automated detection and tracking of sunspots form part of the prediction of solar events and prediction of space weather. In this paper the use of deep learning techniques for automated sunspot detection will be investigated.\n\nKEYWORDS\n\nSunspots, Space Weather, Deep Learning, Object Detection\n\n1. INTRODUCTION\n\nSunspots are dark regions character

In [4]:
!ollama pull nomic-embed-text

pulling manifest â ™ pulling manifest â ™ pulling manifest â ¸ pulling manifest â ¸ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â § pulling manifest â � pulling manifest â ‹ pulling manifest â ‹ pulling manifest â ¹ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ´ pulling manifest 
pulling 970aa74c0a90... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 274 MB                         
pulling c71d239df917... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling ce4a164fc046... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   17 B                         
pulling 31df23ea7daa... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  420 B                         
verifying sha256 digest â ‹ pulling manifest 
pulling 970aa74c0a90... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 274 MB                         
pulling c71d239df917... 100% â–•â–ˆâ–ˆâ–ˆ

In [5]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED     
nomic-embed-text:latest	0a109f422b47	274 MB	1 second ago	
llama2:latest          	78e26419b446	3.8 GB	23 hours ago	


In [7]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [8]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [9]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 3/3 [00:10<00:00,  3.64s/it]


In [10]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [11]:
# LLM from Ollama
local_model = "llama2"
llm = ChatOllama(model=local_model)

In [12]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [13]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
chain.invoke(input(""))

 How can I easily implement this paper?


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.16s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


'To implement the paper, you will need to follow these steps:\n\n1. Familiarize yourself with the paper: Before implementing the paper, read it carefully to understand the authors\' methodology and findings.\n2. Install necessary libraries and tools: Depending on the specific implementation, you may need to install certain libraries or tools to perform object detection or text classification. Some common libraries used in deep learning include TensorFlow, PyTorch, and Keras.\n3. Collect and preprocess data: The authors of the paper use a dataset of images with sunspots labeled as either "sunspot" or "non-sunspot". You will need to collect a similar dataset for your implementation, or preprocess the data you have to prepare it for training. This may involve resizing images, normalizing pixel values, and splitting the data into training and validation sets.\n4. Train a deep learning model: Use a deep learning framework such as TensorFlow or PyTorch to train a model on your dataset. You w